In [2]:
import pandas as pd

**Importing Data from drive**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
df=pd.read_csv('gdrive/My Drive/dataset/fakenewsclassifier/train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df=df.dropna()

In [7]:
#Independent Feature
X=df.drop('label',axis=1)

In [8]:
#Dependent features
y=df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

# PRE-PROCESSING

### Onehot Representation

In [11]:
messages=X.copy()

In [12]:
messages.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Stemming

In [15]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
voc_size=5000
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

### Embedding Representation

In [19]:
# Padding
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4266 3668 1819]
 [   0    0    0 ... 2810  905 3676]
 [   0    0    0 ... 4590  777  514]
 ...
 [   0    0    0 ... 1331 4780 4220]
 [   0    0    0 ... 3074 2396 4563]
 [   0    0    0 ... 1464 2379 4984]]


In [24]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

#LSTM

In [28]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 53ms/step - loss: 0.4713 - accuracy: 0.7449 - val_loss: 0.1983 - val_accuracy: 0.9185
Epoch 2/10
192/192 [==============================] - 9s 48ms/step - loss: 0.1260 - accuracy: 0.9523 - val_loss: 0.1958 - val_accuracy: 0.9193
Epoch 3/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0810 - accuracy: 0.9729 - val_loss: 0.2276 - val_accuracy: 0.9115
Epoch 4/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0492 - accuracy: 0.9841 - val_loss: 0.3323 - val_accuracy: 0.9183
Epoch 5/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0318 - accuracy: 0.9896 - val_loss: 0.2684 - val_accuracy: 0.9117
Epoch 6/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0258 - accuracy: 0.9923 - val_loss: 0.4194 - val_accuracy: 0.9125
Epoch 7/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0110 - accuracy: 0.9964 - val_loss: 0.4319 - val_accuracy: 0.911

### Performance Metrics And Accuracy

In [30]:

y_pred1=model1.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [31]:
from sklearn.metrics import confusion_matrix

In [32]:
confusion_matrix(y_test,y_pred1)

array([[3103,  316],
       [ 239, 2377]])

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.908036454018227

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

